Universidad del Valle de Guatemala <br>
Security Data Science <br>
Jorge Yass <br>
Douglas de León - 18037 <br>

# Laboratorio 6 - Hunting with Data Science

## Parte 1 - Filtrado y preprocesamiento

In [2]:
import json

import pandas as pd

In [3]:
data = []
with open("large_eve.json") as file:
    for line in file.readlines():
        data.append(json.loads(line))

In [4]:
print("Cantidad de registros:", "{:,}".format(len(data)))

Cantidad de registros: 746,909


In [5]:
only_dns = list(filter(lambda x: "dns" in x, data))

In [6]:
print("Cantidad de registros DNS: ", "{:,}".format(len(only_dns)))

Cantidad de registros DNS:  15,749


In [7]:
print("Ejemplos: \n")
for ej in only_dns[:2]:
    print(json.dumps(ej, indent=4))

Ejemplos: 

{
    "timestamp": "2017-07-22T17:33:16.661646-0500",
    "flow_id": 1327836194150542,
    "pcap_cnt": 22269,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
    "src_port": 59680,
    "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 15529,
        "rrname": "api.wunderground.com",
        "rrtype": "A",
        "tx_id": 0
    }
}
{
    "timestamp": "2017-07-22T17:33:24.990320-0500",
    "flow_id": 2022925111925872,
    "pcap_cnt": 54352,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
    "src_port": 38051,
    "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 58278,
        "rrname": "stork79.dropbox.com",
        "rrtype": "A",
        "tx_id": 0
    }
}


In [8]:
df = pd.json_normalize(data=only_dns)
df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


In [9]:
A_type = df[df['dns.rrtype'] == 'A']
print(len(A_type))
A_type.head()

2849


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
5,2017-07-22T17:33:36.672785-0500,237919524635665,55496,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,41663,2001:07fd:0000:0000:0000:0000:0000:0001,53,UDP,query,45082,api.wunderground.com,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN


In [10]:
A_type['dns.rrname'].unique()

array(['api.wunderground.com', 'stork79.dropbox.com',
       'hpca-tier2.office.aol.com.ad.aol.aoltw.net',
       'safebrowsing.clients.google.com.home', 'fxfeeds.mozilla.com',
       'www.metasploit.com', 'aolmtcmxm03.office.aol.com',
       'aolmtcmxm02.office.aol.com.ad.aol.aoltw.net',
       'aolmtcmxm02.office.aol.com', 'hpca-tier2.office.aol.com',
       'aolmtcmxm03.office.aol.com.ad.aol.aoltw.net',
       'aolmtcmxm04.office.aol.com', 'safebrowsing.clients.google.com',
       'wpad.home', 'safebrowsing.clients.google.com.stayonline.net',
       'aolmtcmxm04.office.aol.com.ad.aol.aoltw.net',
       'AOLDTCMA04.ad.aol.aoltw.net.office.aol.com',
       'AOLDTCMA04.office.aol.com', '192.168.22.110phpmyadmin',
       'secure.informaction.com', 'secure.informaction.com.localdomain',
       'safebrowsing.clients.google.com.localdomain', 'ueip.vmware.com',
       'en-us.fxfeeds.mozilla.com',
       '192.168.22.110phpmyadmin.localdomain', 'time.windows.com',
       'softwareupdate.vmwar

In [11]:
from flare.data_science.features import domain_tld_extract

for domain in A_type['dns.rrname'].unique():
    print(domain_tld_extract(domain))

wunderground.com
dropbox.com
aoltw.net
safebrowsing.clients.google.com.home
mozilla.com
metasploit.com
aol.com
aoltw.net
aol.com
aol.com
aoltw.net
aol.com
google.com
wpad.home
stayonline.net
aoltw.net
aol.com
aol.com
192.168.22.110phpmyadmin
informaction.com
secure.informaction.com.localdomain
safebrowsing.clients.google.com.localdomain
vmware.com
mozilla.com
192.168.22.110phpmyadmin.localdomain
windows.com
vmware.com
ntkrnlpa.info
portswigger.net
offensive-security.com
stayonline.net
stopbadware.org
aoltw.net
arrancar.org
sql-ledger.org
backtrack-linux.org
stayonline.net
mozilla.com
theanime.cn
theanime.cn
aoltw.net
aoltw.net
aoltw.net
safebrowsing.clients.google.com.hackerlabs.vpn
stayonline.net
aoltw.net
aol.com
aoltw.net
phpmyadmin.net
google.com
google.com
microsoft.com
secure.informaction.com.home
aoltw.net
comcast.net
google.com
aoltw.net
ubuntu.com
mozilla.com
alexa.com
postgresql.org
sourceforge.net
freepbx.org
secure.informaction.com.hackerlabs.vpn
bigflickrfeed.com
gnu.org
w

## Parte 2 - Data Science

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import features_utils as fu

In [ ]:
dataset = A_type

In [ ]:
y = dataset["status"]
X = dataset

In [ ]:
dataset['f1'] = dataset['url'].map(fu.f1) # Full URL length.
dataset['f2'] = dataset['url'].map(fu.f2) # Hostname length.
dataset['f4'] = dataset['url'].map(fu.f4) # Number of occurences of character '.' in URL.
dataset['f5'] = dataset['url'].map(fu.f5) # Number of occurences of character '-' in URL.
dataset['f6'] = dataset['url'].map(fu.f6) # Number of occurences of character '@' in URL.
dataset['f7'] = dataset['url'].map(fu.f7) # Number of occurences of character '?' in URL.
dataset['f8'] = dataset['url'].map(fu.f8) # Number of occurences of character '&' in URL.
dataset['f9'] = dataset['url'].map(fu.f9) # Number of occurences of character '|' in URL.
dataset['f10'] = dataset['url'].map(fu.f10) # Number of occurences of character '=' in URL.
dataset['f11'] = dataset['url'].map(fu.f11) # Number of occurences of character '_' in URL.
dataset['f12'] = dataset['url'].map(fu.f12) # Number of occurences of character '~' in URL.
dataset['f13'] = dataset['url'].map(fu.f13) # Number of occurences of character '%' in URL.
dataset['f14'] = dataset['url'].map(fu.f14) # Number of occurences of character '/' in URL.
dataset['f15'] = dataset['url'].map(fu.f15) # Number of occurences of character '*' in URL.
dataset['f16'] = dataset['url'].map(fu.f16) # Number of occurences of character ':' in URL.
dataset['f17'] = dataset['url'].map(fu.f17) # Number of occurences of character ',' in URL.
dataset['f18'] = dataset['url'].map(fu.f18) # Number of occurences of character ';' in URL.
dataset['f19'] = dataset['url'].map(fu.f19) # Number of occurences of character '$' in URL.
dataset['f20'] = dataset['url'].map(fu.f20) # Number of occurences of character '%20' or space in URL.
dataset['f25'] = dataset['url'].map(fu.f25) # Checks for HTTPS token in URL.
dataset['f26'] = dataset['url'].map(fu.f26) # Ratio of digits in full URL.
dataset['f27'] = dataset['url'].map(fu.f27) # Ratio of digits in hostname.

In [ ]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [ ]:
# Entrenamos el modelo
rf.fit(X_train, y_train)

In [ ]:
# Realizamos predicciones
y_preds = rf.predict(X_test)


In [ ]:
classification_report(y_test, y_preds)

## Parte 3 - Dominio de experto